### Install required packages

!pip install yfinance

!pip install pandas-datareader

#### Import dependencies

In [1]:
from pandas_datareader import data
import yfinance
import pandas as pd
import sqlite3
import warnings
warnings.simplefilter(action="ignore")

In [2]:
yfinance.pdr_override()

## This is the original data before ETL
## This is the Extract Part

In [3]:
Apple_df=data.get_data_yahoo("AAPL",start="2023-01-01", end="2023-12-31")
display(Apple_df.head())
display(Apple_df.tail())

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-03,130.279999,130.899994,124.169998,125.070000,124.374802,112117500
2023-01-04,126.889999,128.660004,125.080002,126.360001,125.657639,89113600
2023-01-05,127.129997,127.769997,124.760002,125.019997,124.325073,80962700
2023-01-06,126.010002,130.289993,124.889999,129.619995,128.899506,87754700
2023-01-09,130.470001,133.410004,129.889999,130.149994,129.426575,70790800


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-12-22,195.179993,195.410004,192.970001,193.600006,193.600006,37122800
2023-12-26,193.610001,193.889999,192.830002,193.050003,193.050003,28919300
2023-12-27,192.490005,193.500000,191.089996,193.149994,193.149994,48087700
2023-12-28,194.139999,194.660004,193.169998,193.580002,193.580002,34049900
2023-12-29,193.899994,194.399994,191.729996,192.529999,192.529999,42628800


## This is the Transform Part

In [4]:
Apple_volume=Apple_df[["Adj Close","Volume"]]
Apple_volume["stock_id"]="AAPL"
Apple_volume.reset_index(inplace=True)
Apple_volume=Apple_volume[["Date","stock_id","Adj Close","Volume"]]
Apple_volume

,Date,stock_id,Adj Close,Volume
0,2023-01-03,AAPL,124.374802,112117500
1,2023-01-04,AAPL,125.657639,89113600
2,2023-01-05,AAPL,124.325073,80962700
3,2023-01-06,AAPL,128.899506,87754700
4,2023-01-09,AAPL,129.426575,70790800
...,...,...,...,...
245,2023-12-22,AAPL,193.600006,37122800
246,2023-12-26,AAPL,193.050003,28919300
247,2023-12-27,AAPL,193.149994,48087700
248,2023-12-28,AAPL,193.580002,34049900


In [5]:
Apple_df.drop(columns=["Adj Close","Volume"],inplace=True)
Apple_df.head()

,Open,High,Low,Close
Date,,,,
2023-01-03,130.279999,130.899994,124.169998,125.070000
2023-01-04,126.889999,128.660004,125.080002,126.360001
2023-01-05,127.129997,127.769997,124.760002,125.019997
2023-01-06,126.010002,130.289993,124.889999,129.619995
2023-01-09,130.470001,133.410004,129.889999,130.149994


In [6]:
Apple_df[["Open","High","Low","Close"]]=Apple_df[["Open","High","Low","Close"]].apply(lambda x:round(x,2))

In [7]:
Apple_df["Percentage_change"]=Apple_df["Close"].pct_change()

In [8]:
Apple_df.reset_index(inplace=True)


In [9]:
Apple_df["month"]=Apple_df["Date"].dt.month

In [10]:
Apple_df["normalization"]=Apple_df["Close"]/125.07
Apple_df["stock_id"]="AAPL"


In [11]:
Apple_df["Accumulation"]=(Apple_df["Percentage_change"]+1).cumprod()

In [12]:
Apple_df[["normalization","Percentage_change","Accumulation"]]=Apple_df[["normalization","Percentage_change","Accumulation"]].apply(lambda x:round(x,4))

In [13]:
Apple_df=Apple_df[["Date","stock_id","Open","High","Low","Close","month","normalization","Percentage_change","Accumulation"]]

In [14]:
Apple_df

,Date,stock_id,Open,High,Low,Close,month,normalization,Percentage_change,Accumulation
0,2023-01-03,AAPL,130.28,130.90,124.17,125.07,1,1.0000,NaN,NaN
1,2023-01-04,AAPL,126.89,128.66,125.08,126.36,1,1.0103,0.0103,1.0103
2,2023-01-05,AAPL,127.13,127.77,124.76,125.02,1,0.9996,-0.0106,0.9996
3,2023-01-06,AAPL,126.01,130.29,124.89,129.62,1,1.0364,0.0368,1.0364
4,2023-01-09,AAPL,130.47,133.41,129.89,130.15,1,1.0406,0.0041,1.0406
...,...,...,...,...,...,...,...,...,...,...
245,2023-12-22,AAPL,195.18,195.41,192.97,193.60,12,1.5479,-0.0055,1.5479
246,2023-12-26,AAPL,193.61,193.89,192.83,193.05,12,1.5435,-0.0028,1.5435
247,2023-12-27,AAPL,192.49,193.50,191.09,193.15,12,1.5443,0.0005,1.5443
248,2023-12-28,AAPL,194.14,194.66,193.17,193.58,12,1.5478,0.0022,1.5478


### After reviewing Apple Stock Data, create a function for other stocks

In [15]:
def stock_to_frame(Ticker, start="2023-01-01",end="2023-12-31"):
    """this is a function take a parameter ticker and return two dataframes for that stock
       , the two dataframes are saved in the list, list[0]is the price dataframe, list[1] is 
       the volume dataframe. 
    """
    temp_df=data.get_data_yahoo(Ticker,start=start, end=end)
    temp_df[["Open","High","Low","Close"]]=temp_df[["Open","High","Low","Close"]].apply(lambda x:round(x,2))
    volume=temp_df[["Adj Close","Volume"]]
    volume["stock_id"]=Ticker
    volume.reset_index(inplace=True)
    volume=volume[["Date","stock_id","Adj Close","Volume"]]
    temp_df.drop(columns=["Adj Close","Volume"],inplace=True)
    temp_df["Percentage_change"]=temp_df["Close"].pct_change()
    temp_df.reset_index(inplace=True)
    temp_df["month"]=temp_df["Date"].dt.month
    temp_df["normalization"]=temp_df["Close"]/temp_df["Close"][0]
    temp_df["stock_id"]=Ticker
    temp_df["Accumulation"]=(temp_df["Percentage_change"]+1).cumprod()
    temp_df[["normalization","Percentage_change","Accumulation"]]=temp_df[["normalization","Percentage_change","Accumulation"]].apply(lambda x:round(x,4))
    temp_df=temp_df[["Date","stock_id","Open","High","Low","Close","month","normalization","Percentage_change","Accumulation"]]
    
    return temp_df,volume

In [16]:
Google_df=stock_to_frame("GOOGL")

[*********************100%%**********************]  1 of 1 completed


In [17]:
Google_df[0]

,Date,stock_id,Open,High,Low,Close,month,normalization,Percentage_change,Accumulation
0,2023-01-03,GOOGL,89.59,91.05,88.52,89.12,1,1.0000,NaN,NaN
1,2023-01-04,GOOGL,90.35,90.65,87.27,88.08,1,0.9883,-0.0117,0.9883
2,2023-01-05,GOOGL,87.47,87.57,85.90,86.20,1,0.9672,-0.0213,0.9672
3,2023-01-06,GOOGL,86.79,87.69,84.86,87.34,1,0.9800,0.0132,0.9800
4,2023-01-09,GOOGL,88.36,90.05,87.86,88.02,1,0.9877,0.0078,0.9877
...,...,...,...,...,...,...,...,...,...,...
245,2023-12-22,GOOGL,140.77,141.99,140.71,141.49,12,1.5876,0.0076,1.5876
246,2023-12-26,GOOGL,141.59,142.68,141.19,141.52,12,1.5880,0.0002,1.5880
247,2023-12-27,GOOGL,141.59,142.08,139.89,140.37,12,1.5751,-0.0081,1.5751
248,2023-12-28,GOOGL,140.78,141.14,139.75,140.23,12,1.5735,-0.0010,1.5735


In [18]:
Google_df[1]

,Date,stock_id,Adj Close,Volume
0,2023-01-03,GOOGL,89.120003,28131200
1,2023-01-04,GOOGL,88.080002,34854800
2,2023-01-05,GOOGL,86.199997,27194400
3,2023-01-06,GOOGL,87.339996,41381500
4,2023-01-09,GOOGL,88.019997,29003900
...,...,...,...,...
245,2023-12-22,GOOGL,141.490005,26514600
246,2023-12-26,GOOGL,141.520004,16780300
247,2023-12-27,GOOGL,140.369995,19628600
248,2023-12-28,GOOGL,140.229996,16045700


In [19]:
Intel_df=stock_to_frame("INTC")

[*********************100%%**********************]  1 of 1 completed


In [20]:
Intel_df[0]

,Date,stock_id,Open,High,Low,Close,month,normalization,Percentage_change,Accumulation
0,2023-01-03,INTC,27.05,27.07,26.37,26.73,1,1.0000,NaN,NaN
1,2023-01-04,INTC,27.41,27.92,27.16,27.68,1,1.0355,0.0355,1.0355
2,2023-01-05,INTC,27.44,27.78,27.38,27.56,1,1.0311,-0.0043,1.0311
3,2023-01-06,INTC,27.93,28.83,27.38,28.73,1,1.0748,0.0425,1.0748
4,2023-01-09,INTC,28.84,29.87,28.84,29.31,1,1.0965,0.0202,1.0965
...,...,...,...,...,...,...,...,...,...,...
245,2023-12-22,INTC,47.25,48.16,47.20,48.00,12,1.7957,0.0195,1.7957
246,2023-12-26,INTC,48.92,50.52,48.71,50.50,12,1.8893,0.0521,1.8893
247,2023-12-27,INTC,50.63,51.28,50.19,50.76,12,1.8990,0.0051,1.8990
248,2023-12-28,INTC,50.81,50.87,50.16,50.39,12,1.8851,-0.0073,1.8851


In [21]:
Microsoft_df=stock_to_frame("MSFT")

[*********************100%%**********************]  1 of 1 completed


In [22]:
Microsoft_df[0]

,Date,stock_id,Open,High,Low,Close,month,normalization,Percentage_change,Accumulation
0,2023-01-03,MSFT,243.08,245.75,237.40,239.58,1,1.0000,NaN,NaN
1,2023-01-04,MSFT,232.28,232.87,225.96,229.10,1,0.9563,-0.0437,0.9563
2,2023-01-05,MSFT,227.20,227.55,221.76,222.31,1,0.9279,-0.0296,0.9279
3,2023-01-06,MSFT,223.00,225.76,219.35,224.93,1,0.9389,0.0118,0.9389
4,2023-01-09,MSFT,226.45,231.24,226.41,227.12,1,0.9480,0.0097,0.9480
...,...,...,...,...,...,...,...,...,...,...
245,2023-12-22,MSFT,373.68,375.18,372.71,374.58,12,1.5635,0.0028,1.5635
246,2023-12-26,MSFT,375.00,376.94,373.50,374.66,12,1.5638,0.0002,1.5638
247,2023-12-27,MSFT,373.69,375.06,372.81,374.07,12,1.5614,-0.0016,1.5614
248,2023-12-28,MSFT,375.37,376.46,374.16,375.28,12,1.5664,0.0032,1.5664


In [23]:
Oracle_df=stock_to_frame("ORCL")

[*********************100%%**********************]  1 of 1 completed


In [24]:
Oracle_df[0]

,Date,stock_id,Open,High,Low,Close,month,normalization,Percentage_change,Accumulation
0,2023-01-03,ORCL,82.47,83.84,82.26,83.72,1,1.0000,NaN,NaN
1,2023-01-04,ORCL,84.20,85.17,83.64,84.48,1,1.0091,0.0091,1.0091
2,2023-01-05,ORCL,85.00,85.40,83.22,84.31,1,1.0070,-0.0020,1.0070
3,2023-01-06,ORCL,84.74,86.34,83.82,85.66,1,1.0232,0.0160,1.0232
4,2023-01-09,ORCL,86.84,87.59,86.16,86.42,1,1.0323,0.0089,1.0323
...,...,...,...,...,...,...,...,...,...,...
245,2023-12-22,ORCL,105.70,106.49,105.46,106.20,12,1.2685,0.0032,1.2685
246,2023-12-26,ORCL,106.52,106.65,105.34,106.19,12,1.2684,-0.0001,1.2684
247,2023-12-27,ORCL,106.32,106.33,105.56,105.94,12,1.2654,-0.0024,1.2654
248,2023-12-28,ORCL,106.01,106.61,105.58,106.25,12,1.2691,0.0029,1.2691


# Create a Sqlite Database and store data in it

In [25]:
db_name="Group_Project_3.db"
engine=sqlite3.connect(db_name)

In [26]:
query="""drop table if exists stock"""
engine.execute(query)
query="""
create table stock
(
    Stock_id int, 
    Stock_name varchar(20) not null,
    Ticker varchar(10) not null,
    primary key (Stock_id)
)"""

engine.execute(query)

In [27]:
query="""
insert into stock values
(1, 'APPLE', 'APPL'), 
(2, 'GOOGLE','GOOGL'),
(3, 'INTEL', 'INTC'), 
(4, 'MICROSOFT','MSFT'),
(5, 'ORACLE', 'ORCL') 
"""
engine.execute(query)

In [28]:
engine.commit()

In [29]:
def create_stock_table (table_name):
    query=f"""drop table if exists {table_name}"""
    engine.execute(query)
    query=f"""
    create table {table_name}
    (
        date timestamp,
        stock_id varchar(10),
        Open real not null, 
        High real not null, 
        Low real not null, 
        Close real not null, 
        Month int not null,
        normalization real not null,
        Percentage_change real, 
        accumulation real,
        primary key (date, stock_id),
        constraint fk_stock,
        foreign key(stock_id) references stock(Ticker)
    )
    """
    engine.execute(query)

In [30]:
create_stock_table ("Apple_price")

In [31]:
Apple_df.to_sql("Apple_price", con=engine,if_exists="append",index=False)

250

In [32]:
def create_volume_table (table_name):
    query=f"""drop table if exists {table_name}"""
    engine.execute(query)
    query=f"""
    create table {table_name}
    (
        date timestamp,
        stock_id varchar(10),
        'Adj Close' real not null, 
        Volume int not null, 
        primary key (date, stock_id),
        constraint fk_stock,
        foreign key(stock_id) references {table_name}(stock_id),
        foreign key(date) references {table_name}(date)
    )
    """
    engine.execute(query)

In [33]:
create_volume_table("Apple_volume")

In [34]:
Apple_volume.to_sql("Apple_volume",con=engine,if_exists="append",index=False)

250

In [35]:
create_stock_table ("Google_price")
Google_df[0].to_sql("Google_price", con=engine,if_exists="append",index=False)
create_volume_table("Google_volume")
Google_df[1].to_sql("Google_volume",con=engine,if_exists="append",index=False)

250

In [36]:
create_stock_table ("Microsoft_price")
Microsoft_df[0].to_sql("Microsoft_price", con=engine,if_exists="append",index=False)
create_volume_table("Microsoft_volume")
Microsoft_df[1].to_sql("Microsoft_volume",con=engine,if_exists="append",index=False)

250

In [37]:
create_stock_table ("Intel_price")
Intel_df[0].to_sql("Intel_price", con=engine,if_exists="append",index=False)
create_volume_table("Intel_volume")
Intel_df[1].to_sql("Intel_volume",con=engine,if_exists="append",index=False)

250

In [38]:
create_stock_table ("Oracle_price")
Oracle_df[0].to_sql("Oracle_price", con=engine,if_exists="append",index=False)
create_volume_table("Oracle_volume")
Oracle_df[1].to_sql("Oracle_volume",con=engine,if_exists="append",index=False)

250